<a href="https://colab.research.google.com/github/annvix/Accents_LSTM-/blob/master/Train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!git clone https://github.com/annvix/Accents_LSTM-.git

Cloning into 'Accents_LSTM-'...
remote: Enumerating objects: 20, done.
remote: Counting objects: 100% (20/20), done.
remote: Compressing objects: 100% (17/17), done.
remote: Total 20 (delta 6), reused 17 (delta 3), pack-reused 0
Unpacking objects: 100% (20/20), done.


In [0]:
import numpy as np
# read data from dataset
with open('Accents_LSTM-/Data.csv', 'r') as f:
#with open('Data.csv', 'r') as f:
    all_accents = f.readlines()
data_len = len(all_accents)

In [0]:
words, accents = [], []

In [0]:
for i in range(data_len):
    str = all_accents[i].replace('\n', '').split(',')
    words.append(str[0])
    accents.append(str[1])
max_len = len(max(words, key=len))

In [4]:
print('Max word length:')
print(max_len)

Max word length:
30


In [0]:
words_int = np.zeros((data_len,max_len), dtype=int)
accents_int = np.zeros(data_len, dtype=int)

In [0]:
for i in range(data_len):
    for j in range(len(words[i])):
        words_int[i][j]=ord(words[i][j])
    accents_int[i] = accents[i].find(chr(39)) + 1

In [7]:
print('Words without accents:')
for i in range(len(words)):
    if accents_int[i]==0:
        print(words[i])

Words without accents:
б
брр
в
гм
ж
к
кш
ль
мм
с
тсс
хм


In [0]:
train_frac = 0.8
val_frac = 0.1
test_frac = 0.1


def train_test_val_words(words_int):
    return (words_int[:int(len(words_int)*train_frac)], 
words_int[int(len(words_int)*train_frac) : (int(len(words_int)*train_frac) + int(len(words_int)*val_frac))], 
words_int[(int(len(words_int)*train_frac) + int(len(words_int)*val_frac)):])
    
def train_test_val_accents(accents_int):
    return (accents_int[:int(len(accents_int)*train_frac)], 
accents_int[int(len(accents_int)*train_frac) : (int(len(accents_int)*train_frac) + int(len(accents_int)*val_frac))], 
accents_int[(int(len(accents_int)*train_frac) + int(len(accents_int)*val_frac)):])

train_x, val_x, test_x = train_test_val_words(words_int)
train_y, val_y, test_y = train_test_val_accents(accents_int)

In [9]:
## print out the shapes of your resultant feature data
print("\t\t\t Sizes:")
print("Train set: \t\t{}".format(train_x.shape), 
      "\nValidation set: \t{}".format(val_x.shape),
      "\nTest set: \t\t{}".format(test_x.shape))

			 Sizes:
Train set: 		(1343301, 30) 
Validation set: 	(167912, 30) 
Test set: 		(167914, 30)


In [10]:
print("\t\t Vizualization:")
train = ''
for i in range(10):
    train = ''
    for j in range(len(train_x[i])):
      if (train_x[i][j] == 0):
        break
      train = train + chr(train_x[i][j])
    print("Training set: ",train)
for i in range(10):    
    vval = ''
    for j in range(len(val_x[i])):
      if (val_x[i][j] == 0):
        break
      vval = vval + chr(val_x[i][j])
    print("Validation set: ",vval)
for i in range(10):
    test = ''
    for j in range(len(test_x[i])):
      if (test_x[i][j] == 0):
        break
      test = test + chr(test_x[i][j])
    print("Test set: ",test)


		 Vizualization:
Training set:  а
Training set:  аав
Training set:  аава
Training set:  аавам
Training set:  аавами
Training set:  аавасакса
Training set:  аавах
Training set:  ааве
Training set:  аавик
Training set:  аавов
Validation set:  самоокупитесь
Validation set:  самоокупится
Validation set:  самоокупиться
Validation set:  самоокупишься
Validation set:  самоокуплюсь
Validation set:  самоокупятся
Validation set:  самооплодотворение
Validation set:  самооплодотворением
Validation set:  самооплодотворении
Validation set:  самооплодотворений
Test set:  трудоустройством
Test set:  трудоустройству
Test set:  трудоустройся
Test set:  трудоустроит
Test set:  трудоустроите
Test set:  трудоустройте
Test set:  трудоустроитесь
Test set:  трудоустройтесь
Test set:  трудоустроится
Test set:  трудоустроить


In [0]:
import torch
from torch.utils.data import TensorDataset, DataLoader

# create Tensor datasets for train, test and val
train_data = TensorDataset(torch.from_numpy(train_x), torch.from_numpy(train_y))
valid_data = TensorDataset(torch.from_numpy(val_x), torch.from_numpy(val_y))
test_data = TensorDataset(torch.from_numpy(test_x), torch.from_numpy(test_y))

# dataloaders
batch_size = 50 

# make sure to SHUFFLE your training data. Keep Shuffle=True.
train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size)
valid_loader = DataLoader(valid_data, shuffle=True, batch_size=batch_size)
test_loader = DataLoader(test_data, shuffle=True, batch_size=batch_size)

In [12]:
print(type(train_data))
print(type(train_x))

<class 'torch.utils.data.dataset.TensorDataset'>
<class 'numpy.ndarray'>


In [13]:
# obtain one batch of training data and label. 
dataiter = iter(train_loader)
sample_x, sample_y = dataiter.next()

print('Sample input size: ', sample_x.size()) # batch_size, seq_length
print('Sample input: \n', sample_x)
print()
print('Sample label size: ', sample_y.size()) # batch_size
print('Sample label: \n', sample_y)

Sample input size:  torch.Size([50, 30])
Sample input: 
 tensor([[1074, 1082, 1088,  ...,    0,    0,    0],
        [1088, 1077, 1081,  ...,    0,    0,    0],
        [1074, 1077, 1088,  ...,    0,    0,    0],
        ...,
        [1088, 1072, 1079,  ...,    0,    0,    0],
        [1088, 1086, 1076,  ...,    0,    0,    0],
        [1088, 1072, 1079,  ...,    0,    0,    0]])

Sample label size:  torch.Size([50])
Sample label: 
 tensor([ 4,  2,  6,  3,  5,  8,  8,  4,  5,  3,  4, 10,  6,  6,  4,  6,  6,  6,
         7,  8,  6,  6,  7,  2,  6,  7,  4, 10,  6,  4,  3, 12,  7,  7,  6,  8,
         7,  9,  5,  6,  6,  4,  6,  5,  6,  7,  6,  8,  7,  5])


In [14]:
# Check if GPU is available.
train_on_gpu=torch.cuda.is_available()

if(train_on_gpu):
    print('Training on GPU.')
else:
    print('No GPU available, training on CPU.')

Training on GPU.


In [0]:
import torch.nn as nn

class SentimentLSTM(nn.Module):

    def __init__(self, data_len, output_size, embedding_dim, hidden_dim, n_layers, drop_prob=0.5):

        #Initialize the model by setting up the layers.

        super(SentimentLSTM, self).__init__()

        self.output_size = output_size
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim
        
        # embedding and LSTM layers
        self.embedding = nn.Embedding(data_len, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, n_layers, 
                            dropout=drop_prob, batch_first=True)
        
        # dropout layer
        self.dropout = nn.Dropout(0.3)
        
        # linear and sigmoid layers
        self.fc = nn.Linear(hidden_dim, output_size)
        self.relu = nn.ReLU()
        

    def forward(self, x, hidden):

        #Perform a forward pass of our model on some input and hidden state.

        batch_size = x.size(0)

        # embeddings and lstm_out
        embeds = self.embedding(x)
        lstm_out, hidden = self.lstm(embeds, hidden)
    
        # stack up lstm outputs
        lstm_out = lstm_out.contiguous().view(-1, self.hidden_dim)
        
        # dropout and fully-connected layer
        out = self.dropout(lstm_out)
        out = self.fc(out)
        # relu function
        relu_out = self.relu(out)
        
        # reshape to be batch_size first
        relu_out = relu_out.view(batch_size, -1)
        relu_out = relu_out[:, -1] # get last batch of labels
        
        # return last relu output and hidden state
        return relu_out, hidden
    
    
    def init_hidden(self, batch_size):
        #Initializes hidden state

        # Create two new tensors with sizes n_layers x batch_size x hidden_dim,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
        
        if (train_on_gpu):
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda(),
                      weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_(),
                      weight.new(self.n_layers, batch_size, self.hidden_dim).zero_())
        
        return hidden
    

In [16]:
# Instantiate the model with these hyperparameters
data_len = data_len + 1 # +1 for the 0 padding + our word tokens
output_size = 1
embedding_dim = 300
hidden_dim = 256
n_layers = 2

net = SentimentLSTM(data_len, output_size, embedding_dim, hidden_dim, n_layers)

print(net)

SentimentLSTM(
  (embedding): Embedding(1679128, 300)
  (lstm): LSTM(300, 256, num_layers=2, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.3, inplace=False)
  (fc): Linear(in_features=256, out_features=1, bias=True)
  (relu): ReLU()
)


In [0]:
# loss and optimization functions
lr=0.001

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=lr)


In [18]:
#Training and Validation

epochs = 4

counter = 0
print_every = 100
clip=5 # gradient clipping

# move model to GPU, if available
if(train_on_gpu):
    net.cuda()

net.train()

SentimentLSTM(
  (embedding): Embedding(1679128, 300)
  (lstm): LSTM(300, 256, num_layers=2, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.3, inplace=False)
  (fc): Linear(in_features=256, out_features=1, bias=True)
  (relu): ReLU()
)

In [23]:
# train for some number of epochs
for e in range(epochs):
    # initialize hidden state
    h = net.init_hidden(batch_size)

    # batch loop
    for inputs, acc_position in train_loader:
        counter += 1

        if(train_on_gpu):
            inputs, acc_position = inputs.cuda(), acc_position.cuda()

        # Creating new variables for the hidden state, otherwise
        # we'd backprop through the entire training history
        h = tuple([each.data for each in h])

        # zero accumulated gradients
        net.zero_grad()

        # get the output from the model
        output, h = net(inputs, h)

        # calculate the loss and perform backprop
        loss = criterion(output.squeeze(), labels.float())
        loss.backward()
        # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
        nn.utils.clip_grad_norm_(net.parameters(), clip)
        optimizer.step()

        # loss stats
        if counter % print_every == 0:
            # Get validation loss
            val_h = net.init_hidden(batch_size)
            val_losses = []
            net.eval()
            for inputs, labels in valid_loader:

                # Creating new variables for the hidden state, otherwise
                # we'd backprop through the entire training history
                val_h = tuple([each.data for each in val_h])

                if(train_on_gpu):
                    inputs, labels = inputs.cuda(), labels.cuda()

                output, val_h = net(inputs, val_h)
                val_loss = criterion(output.squeeze(), labels.float())

                val_losses.append(val_loss.item())

            net.train()
            print("Epoch: {}/{}...".format(e+1, epochs),
                  "Step: {}...".format(counter),
                  "Loss: {:.6f}...".format(loss.item()),
                  "Val Loss: {:.6f}".format(np.mean(val_losses)))

NameError: ignored